#### Lecture 6-2 缺失值及其处理

1.数据缺失原因

* 有些信息暂时无法获取；
* 获取信息的代价太大；
* 信息被遗漏；
* 系统实时性能要求较高；
* 有些对象的某个或某些属性不可用。

2. 缺失值的影响

* 数据分析建模将丢失大量有用信息；
* 数据分析模型所表现出的不确定性更加显著，模型中蕴含的规律更难把握；
* 包含空值的数据会使建模过程陷入混乱，导致不可靠输出。

3. 缺失值的处理方法

* 删除：直接删除相应的属性或样本
* 统计填充：使用所有样本关于这一维度的统计值对其进行填充，包括平均值、中位数、众数、最大值、最小值等
* 统一填充：将所有缺失值统一填充为自定义的值，如空、0、正无穷、负无穷
* 预测填充：通过预测模型利用不存在缺失值的属性来预测缺失值。虽然这种方法比较复杂，但是最后得到的结果比较好。



In [9]:
import numpy as np
import pandas as pd

##### 一、缺失值的统计和删除

统计缺失值可以帮助我们理解数据的缺失情况，删除是数据清洗过程中的重要步骤之一。


1. 缺失数据可以使用`isna`或`isnull`（两个函数没有区别）来查看每个单元格是否缺失，结合`mean`可以计算出每列缺失值的比例：

In [10]:
df = pd.read_csv('data/learn_pandas.csv', usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight', 'Transfer'])
df.isna().head()

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False


In [3]:
df.isna().mean() # 查看缺失的比例

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

In [4]:
# 如果想要查看某一列缺失或者非缺失的行，可以利用`Series`上的`isna`或者`notna`进行布尔索引。例如，查看身高缺失的行：
df[df.Height.isna()].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
12,Senior,Peng You,Female,NaN,48.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N
36,Freshman,Xiaojuan Qin,Male,NaN,79.0,Y
60,Freshman,Yanpeng Lv,Male,NaN,65.0,N


如果想要同时对几个列，检索出全部为缺失或者至少有一个缺失或者没有缺失的行，可以使用`isna, notna`和`any, all`的组合。例如，对身高、体重和转系情况这3列分别进行这三种情况的检索：

In [14]:
sub_set = df[['Height', 'Weight', 'Transfer']]
# sub_set.isna().all(1)
# sub_set.isna().all(1)
df[sub_set.isna().all(1)] # 全部缺失

,Grade,Name,Gender,Height,Weight,Transfer
102,Junior,Chengli Zhao,Male,NaN,NaN,NaN


In [6]:
df[sub_set.isna().any(1)].head() # 至少有一个缺失

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
9,Junior,Juan Xu,Female,164.8,NaN,N
12,Senior,Peng You,Female,NaN,48.0,NaN
21,Senior,Xiaopeng Shen,Male,166.0,62.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N


In [7]:
df[sub_set.notna().all(1)].head() # 没有缺失

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


练习：

请检索出身高、体重全为缺失值的行

2. 缺失信息的删除：pandas提供了dropna()函数来进行删除。

`dropna`的主要参数为

* 轴方向`axis`（默认为0，即删除行）
* 删除方式`how`
* 删除的非缺失值个数阈值`thresh`（$\color{red}{非缺失值}$没有达到这个数量的相应维度会被删除）
* 备选的删除子集`subset`，其中`how`主要有`any`和`all`两种参数可以选择。

例如，删除身高体重至少有一个缺失的行：

In [15]:
res = df.dropna(how = 'any', subset = ['Height', 'Weight'])
res.shape

(174, 6)

In [16]:
res.head()

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


In [17]:
# 删除超过15个缺失值的列
res = df.dropna(1, thresh=df.shape[0]-15) # 身高被删除
res.head()

/var/folders/l_/5lmrjkpx3xv28w5ky6d9w2mc0000gn/T/ipykernel_40573/1099339865.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  res = df.dropna(1, thresh=df.shape[0]-15) # 身高被删除


,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


##### 二、缺失值的填充
##### 1. 利用fillna进行填充

在`fillna`中有三个参数是常用的：`value, method, limit`。
* `value`为填充值，可以是标量，也可以是索引到元素的字典映射；
* `method`为填充方法，有用前面的元素填充`ffill`和用后面的元素填充`bfill`两种类型；
* `limit`参数表示连续缺失值的最大填充次数。

下面构造一个简单的`Series`来说明用法：

In [11]:
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan], list('aaabcd'))
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [12]:
s.fillna(method='ffill') # 用前面的值向后填充

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [13]:
s.fillna(method='bfill') #使用后面的值向前填充

a    1.0
a    1.0
a    2.0
b    2.0
c    2.0
d    NaN
dtype: float64

In [14]:
s.fillna(method='ffill', limit=1) # 连续出现的缺失，最多填充一次

a    NaN
a    1.0
a    1.0
b    NaN
c    2.0
d    2.0
dtype: float64

In [15]:
s.fillna(s.mean()) # value为标量

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [16]:
s.fillna({'a': 100, 'd': 200}) # 通过索引映射填充的值

a    100.0
a      1.0
a    100.0
b      NaN
c      2.0
d    200.0
dtype: float64

In [17]:
# 有时为了更加合理地填充，需要先进行分组后再操作。例如，根据年级进行身高的均值填充：
df.groupby('Grade')['Height'].transform(lambda x: x.fillna(x.mean())).head()

0    158.900000
1    166.500000
2    188.900000
3    163.075862
4    174.000000
Name: Height, dtype: float64

##### 2. 插值函数填充

比较常见的插值法有：线性插值法、近邻插值法、索引插值法

pandas中，插值过程主要由interpolate()函数实现，除了插值方法method（默认为`linear`线性插值）之外，有与`fillna`类似的两个常用参数，一个是控制方向的`limit_direction`，另一个是控制最大连续缺失值插值个数的`limit`。其中，限制插值的方向默认为`forward`，这与`fillna`的`method`中的`ffill`是类似的，若想要后向限制插值或者双向限制插值可以指定为`backward`或`both`。

In [6]:
import pandas as pd
import numpy as np
s = pd.Series([np.nan, np.nan, 1, np.nan, np.nan, np.nan, 2, np.nan, np.nan])
s

0    NaN
1    NaN
2    1.0
3    NaN
4    NaN
5    NaN
6    2.0
7    NaN
8    NaN
dtype: float64

a） 线性插值法

In [7]:
res = s.interpolate(limit_direction='backward', limit=1) #默认线性插值法，进行后向限制插值
res

0     NaN
1    1.00
2    1.00
3     NaN
4     NaN
5    1.75
6    2.00
7     NaN
8     NaN
dtype: float64

In [8]:
res = s.interpolate(limit_direction='both', limit=1) #默认线性插值法，进行双向限制插值
res

0     NaN
1    1.00
2    1.00
3    1.25
4     NaN
5    1.75
6    2.00
7    2.00
8     NaN
dtype: float64

b) 最近邻插值法

In [10]:
s.interpolate('nearest')

0    NaN
1    NaN
2    1.0
3    1.0
4    1.0
5    2.0
6    2.0
7    NaN
8    NaN
dtype: float64

##### 三、课后练习题

1. 缺失数据的筛选

在data/Ex/missing.csv中存放了1000列数据，请按照如下条件进行数据筛选

(1) 选出缺失值比例低于50%的列和缺失值个数超过520的行

(2) 选出连续缺失值个数超过20的列

(3) 若某一列左右两侧的列同时满足行缺失的比例超过10%，则称此列满足缺失对称条件。表中是否存在满足缺失对称条件的列？若存在，请找出所有符合条件的列。

In [7]:
import pandas as pd
import numpy as np
df=pd.read_csv('data/Ex/missing.csv')
#1. 缺失个数大于520的行和缺失比例低于50%的列
df.isna()
# df.isna().sum(1)
# df.isna().mean()
# res=df.loc[df.isna().sum(1)>520,df.isna().mean()<0.5]
# df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f990,f991,f992,f993,f994,f995,f996,f997,f998,f999
0,True,True,True,True,True,True,False,False,True,True,...,False,True,True,False,True,False,True,False,True,False
1,True,False,True,False,True,True,True,True,True,True,...,True,True,True,False,True,False,False,False,False,True
2,True,True,True,False,False,True,True,True,False,True,...,False,True,True,True,True,True,False,False,False,False
3,True,True,True,False,False,True,False,False,True,True,...,False,True,True,False,True,True,True,False,False,True
4,True,False,True,False,False,True,False,False,False,True,...,False,True,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,True,False,True,False,False,True,False,True,False,True,...,True,True,False,False,True,False,True,False,False,True
996,True,False,False,False,True,False,True,True,False,True,...,False,True,False,False,True,False,False,False,False,False
997,True,False,True,False,True,True,False,False,False,True,...,False,True,False,False,True,False,True,False,False,False
998,True,True,True,False,False,True,True,False,False,True,...,False,False,False,False,True,False,False,False,False,True


In [25]:
#2. 选出连续缺失值个数超过20的列
def missing_helper(s):
    temp=s.isna().astype('int').rename('temp_col')
    temp=pd.concat([s,(temp!=temp.shift())],axis=1) #shift 将每一行的值下移一行。
    return temp[s.isna()].groupby('temp_col').size().max()>20

res=df.loc[:,df.apply(missing_helper)]
res.shape



(1000, 977)

In [4]:
#3. 对称条件列
cols=[]
for i in range(1,999):
    temp=df.iloc[:,[i-1,i+1]]
    if temp.isna().all(1).mean()>0.1:
        cols.append('f%d'%i)

len(cols)

677